In [1]:
import pandas as pd
import numpy as np
import csv
import cx_Oracle

In [2]:
df = pd.read_csv("C:\Jypiter\Glob_terror.csv", encoding='latin1', error_bad_lines = False, warn_bad_lines = False, low_memory=False)
df.head(100)

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
5,197001010002,1970,1,1,NaN,0,NaN,217,United States,1,...,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te...",Hewitt Project,-9,-9,0,-9,NaN
6,197001020001,1970,1,2,NaN,0,NaN,218,Uruguay,3,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
7,197001020002,1970,1,2,NaN,0,NaN,217,United States,1,...,"Damages were estimated to be between $20,000-$...",Committee on Government Operations United Stat...,"Christopher Hewitt, ""Political Violence and Te...",NaN,Hewitt Project,-9,-9,0,-9,NaN
8,197001020003,1970,1,2,NaN,0,NaN,217,United States,1,...,The New Years Gang issue a communiqué to a loc...,"Tom Bates, ""Rads: The 1970 Bombing of the Army...","David Newman, Sandra Sutherland, and Jon Stewa...","The Wisconsin Cartographers' Guild, ""Wisconsin...",Hewitt Project,0,0,0,0,NaN
9,197001030001,1970,1,3,NaN,0,NaN,217,United States,1,...,"Karl Armstrong's girlfriend, Lynn Schultz, dro...",Committee on Government Operations United Stat...,"Tom Bates, ""Rads: The 1970 Bombing of the Army...","David Newman, Sandra Sutherland, and Jon Stewa...",Hewitt Project,0,0,0,0,NaN


In [3]:
features = ['iyear', 'imonth', 'iday', 'country_txt', 'region_txt', 'latitude', 'longitude']
df = df.loc[:,features]
df = df.dropna().reset_index(drop=True)
df.loc[:,'accident_date'] = df.astype('str').loc[:,'iyear'] + df.astype('str').loc[:,'imonth'] + df.astype('str').loc[:,'iday']
df.head(100)

,iyear,imonth,iday,country_txt,region_txt,latitude,longitude,accident_date
0,1970,7,2,Dominican Republic,Central America & Caribbean,18.456792,-69.951164,197072
1,1970,0,0,Mexico,North America,19.371887,-99.086624,197000
2,1970,1,0,Philippines,Southeast Asia,15.478598,120.599741,197010
3,1970,1,0,Greece,Western Europe,37.997490,23.762728,197010
4,1970,1,0,Japan,East Asia,33.580412,130.396361,197010
5,1970,1,1,United States,North America,37.005105,-89.176269,197011
6,1970,1,2,Uruguay,South America,-34.891151,-56.187214,197012
7,1970,1,2,United States,North America,37.791927,-122.225906,197012
8,1970,1,2,United States,North America,43.076592,-89.412488,197012
9,1970,1,3,United States,North America,43.072950,-89.386694,197013


In [4]:
con2 = cx_Oracle.connect('SYSTEM/legomania97@//localhost:1521/xe') # логин/пароль@сервер:порт/база данных

#create

def create(con2):
    cur = con2.cursor()
    statement = 'CREATE TABLE podft (iyear varchar2 (20 char),imonth varchar2 (20 char),iday varchar2 (20 char),country_txt varchar2 (200 char),region_txt varchar2 (200 char),latitude varchar2 (20 char),longitude varchar2 (20 char),accident_date varchar2 (20 char))'
    cur.execute(statement)
    con2.commit()

#insert

def insert(con2):
    cur = con2.cursor()
    statement = 'insert into podft values (:2, :3, :4, :5, :6, :7, :8, :9)'
    for index, row in df.iterrows():
        cur.execute(statement, (row['iyear'],row['imonth'],row['iday'],row['country_txt'],row['region_txt'],row['latitude'],row['longitude'],row['accident_date']))
    con2.commit()

#select

def select(con2):
    cur = con2.cursor()
    statement = 'SELECT * FROM podft'
    dt = cur.execute(statement)
    for res in dt:
        print(res)
    data = cur.fetchall()
    print(data)
    con2.commit()

#drop
def drop(con2):
    cur = con2.cursor()
    statement = 'DROP TABLE podft'
    cur.execute(statement)
    con2.commit()
    
create(con2)
insert(con2)
#select(con2)
#drop(con2)

con2.close()

DatabaseError: ORA-00955: name is already used by an existing object

In [29]:
freq = df[['country_txt','latitude']].groupby(['country_txt']).count() #таблица частот
freq = freq.sort_values(by='latitude', ascending=False) #сортируем по возрастанию
freq

KeyError: 'country_txt'

In [24]:
min_i = int(freq.shape[0]*0.1)
max_i = int(freq.shape[0]*0.9)
print(min_i,max_i)

20 183


In [31]:
df2 = freq.iloc[min_i:max_i,:]
df2 = df2[(df2['latitude'] >= 5)]
df2

,latitude
country_txt,
Lebanon,2333
Chile,2320
Libya,2222
West Bank and Gaza Strip,2215
Russia,2191
Syria,2166
Israel,2129
Guatemala,1957
South Africa,1939


In [32]:
print(df2.mean(), df2.var(), df2.std())

latitude    302.006329
dtype: float64 latitude    311783.203781
dtype: float64 latitude    558.375504
dtype: float64
